## fakefriends.csv
1. user_id: 사용자ID
2. name:사용자 이름
3. age: 사용자 나이
4. num_friends: 친구 수

In [1]:
from pyspark import SparkConf, SparkContext
# 스파크 환경 설정 객체 생성
conf = SparkConf().setMaster("local").setAppName("241205_04_RDDEXAM")
spark = SparkContext(conf=conf).getOrCreate()

24/12/05 18:31:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/05 18:31:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/12/05 18:31:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [2]:
import os
directory = os.path.join(os.getcwd(), "data")
filename = "fakefriends.csv"
filepath = os.path.join(directory, filename)

lines = spark.textFile("file:///"+filepath.replace("\\", "/"))
lines.take(5)

['0,Will,33,385',
 '1,Jean-Luc,26,2',
 '2,Hugh,55,221',
 '3,Deanna,40,465',
 '4,Quark,68,21']

In [5]:
name_age_friends = lines.map(lambda x: x.split(','))
name_age_friends.take(5)

[['0', 'Will', '33', '385'],
 ['1', 'Jean-Luc', '26', '2'],
 ['2', 'Hugh', '55', '221'],
 ['3', 'Deanna', '40', '465'],
 ['4', 'Quark', '68', '21']]

In [6]:
# 평균 친구수 구하기

In [7]:
total_people = name_age_friends.count()
total_people

500

In [8]:
friends_cnt = name_age_friends.map(lambda x:x[3])
friends_cnt.take(3)


['385', '2', '221']

In [9]:
total_friend = friends_cnt.map(int).sum()
total_friend

124266

In [10]:
avg_friend = total_friend / total_people
avg_friend

248.532

In [11]:
# 가장 친구가 많은/적은 사용자 찾기

In [12]:
inssa = name_age_friends.sortBy(lambda x: int(x[3]),  ascending=False)
inssa.take(10)

[['106', 'Beverly', '18', '499'],
 ['278', 'Data', '44', '499'],
 ['353', 'Julian', '64', '499'],
 ['192', 'Jadzia', '45', '497'],
 ['266', 'Jean-Luc', '66', '496'],
 ['408', 'Dukat', '62', '496'],
 ['273', 'Data', '51', '493'],
 ['430', 'Geordi', '36', '493'],
 ['89', 'Worf', '24', '492'],
 ['280', 'Nerys', '26', '492']]

In [13]:
asssa = name_age_friends.sortBy(lambda x: int(x[3]),  ascending=True)
asssa.take(10)

[['24', 'Julian', '25', '1'],
 ['48', 'Nog', '20', '1'],
 ['1', 'Jean-Luc', '26', '2'],
 ['103', 'Ezri', '61', '2'],
 ['194', 'Kasidy', '38', '2'],
 ['491', 'Rom', '60', '2'],
 ['336', 'Gowron', '47', '4'],
 ['54', 'Brunt', '19', '5'],
 ['215', 'Will', '22', '6'],
 ['452', 'Ezri', '58', '6']]

In [14]:
# 친구 수가 100 이상인 사용자 필터링

In [15]:
from py4j.protocol import Py4JJavaError

In [16]:
inssa = name_age_friends.filter(lambda x: 200> int(x[3]) >= 100).sortBy(lambda x: int(x[3]),  ascending=False)
inssa.take(10)

[['227', 'Ezri', '40', '198'],
 ['33', 'Dukat', '53', '197'],
 ['56', 'Gowron', '44', '194'],
 ['467', 'Weyoun', '42', '194'],
 ['494', 'Kasidy', '18', '194'],
 ['139', 'Nog', '36', '193'],
 ['39', 'Morn', '31', '192'],
 ['10', 'Odo', '53', '191'],
 ['310', 'Odo', '68', '189'],
 ['88', 'Ben', '66', '188']]

In [17]:
# 특정 나이에 속한 사람들의 친구 수 합계 , 50대

In [18]:
name_age_friends.take(3)

[['0', 'Will', '33', '385'],
 ['1', 'Jean-Luc', '26', '2'],
 ['2', 'Hugh', '55', '221']]

In [19]:
fifty_friends = name_age_friends.filter(lambda x: 50 <= int(x[2]) <= 59)
fifty_friends.take(3)

[['2', 'Hugh', '55', '221'],
 ['5', 'Weyoun', '59', '318'],
 ['7', 'Will', '54', '307']]

In [20]:
fifty_total = fifty_friends.map(lambda x: int(x[3])).sum()
fifty_total

24360

In [21]:
# 다른분꺼

In [29]:
sort_age_info = name_age_friends.map(lambda x: int(x[2]) // 10 ).distinct().sortBy(lambda x: x).collect()
sort_age_info

[1, 2, 3, 4, 5, 6]

In [23]:
#10대부터 60대까지 전체 친구 합계,평균
for i in sort_age_info:
    print(f'''
    {'*'*50}
    {str(i)+'0'}대의 친구 합계 : {name_age_friends.filter(lambda x : (int(x[2])//10) == i).map(lambda x : int(x[3])).sum()}명
    {str(i)+'0'}대의 친구 평균 : {name_age_friends.filter(lambda x : (int(x[2])//10) == i).map(lambda x : int(x[3])).sum() /
    name_age_friends.filter(lambda x : (int(x[2])//10) == i).map(lambda x : int(x[3])).count():.1f}명
    {'*'*50}''')
    


    **************************************************
    10대의 친구 합계 : 5093명
    10대의 친구 평균 : 268.1명
    **************************************************

    **************************************************
    20대의 친구 합계 : 21503명
    20대의 친구 평균 : 231.2명
    **************************************************

    **************************************************
    30대의 친구 합계 : 22887명
    30대의 친구 평균 : 235.9명
    **************************************************

    **************************************************
    40대의 친구 합계 : 26455명
    40대의 친구 평균 : 259.4명
    **************************************************

    **************************************************
    50대의 친구 합계 : 24360명
    50대의 친구 평균 : 259.1명
    **************************************************

    **************************************************
    60대의 친구 합계 : 23968명
    60대의 친구 평균 : 252.3명
    **************************************************


In [26]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(name_age_friends, columns = ['Age','Number of Friends'])
df

ValueError: Shape of passed values is (6, 1), indices imply (6, 2)

In [ ]:
plt.figure(figsize=(8,4))
x = df['Age']
y = df['Number of friends']
plt.bar(x,y)
plt.show()

In [ ]:
# 분석 주제 추가

#같은 이름, 같은 나이 잃어버린 쌍둥이 찾아주기 :) 

# 1.이름과 나이 유저 페어링
pairs_name_age = rdd.map(lambda x: (x[1],x[2]))
#pairs_name_age.collect()

#2. 카운트 
counts = pairs_name_age.map(lambda x: (x,1))
#counts.collect()

#3. 같은 이름 찾기
same_name_age = counts.reduceByKey(lambda x,y: x+y)
#same_name_age.collect()

# 중복 찾기
find_lost_twins = same_name_age.filter(lambda x: x[1] > 1)
find_lost_twins.collect() 

In [ ]:
# 아이디 중복은 없음! - 동일 인물 없음! 
duplicated_userid = rdd.map(lambda x: (x[0],1))
result = duplicated_userid.reduceByKey(lambda x,y:x+y)
result2 = result.filter(lambda x: x[1] > 1)
result2.collect()